# Análise dos Tópicos 

Esse notebook possui os códigos para a análise dos tópicos extraídos dos diários oficiais. 

## Imports Necessários

In [1]:
# Código para poder usar os módulos implementados nesse repositório no notebook 
import sys, os
path_module = os.path.abspath('../modules/')
if path_module not in sys.path:
    sys.path.append(path_module)

In [2]:
# imports do projeto

from utils.process_gazette import ProcessGazette
from preprocess.pre_process_text import PreProcessText
from nlp.extract_topics import ExtractTopics
from nlp.ner import Ner

## Extração e análise dos tópicos do Diário Teste

### Esta seção realizará a filtragem de páginas relacionadas a Contratos Públicos, garantindo apenas conteúdo relevante.

In [24]:
pp = ProcessGazette(BASE_DIR="gazettes/")
txt_files = [f for f in os.listdir("gazettes/") if f.endswith('.txt')]
all_pages = []
all_pages_name = []
for txt_file in txt_files:
    pages = pp.break_pages(txt_file, "ANO [X|V|I]+ ", save_file=False)
    all_pages.append(pages)
    all_pages_name.append(txt_file)

In [25]:
revistas_filtradas = {}
topics_ = {}
ppt = PreProcessText("pt_core_news_lg")

for gazette in range(len(all_pages)):
    revistas_filtradas[gazette] = {}

    for page, text in all_pages[gazette].items():
        topics_[page] = {'text': text}
        tokens_ = ppt.process_text(topics_[page]['text'])
        for token in tokens_:
            if len(tokens_) > 5 and page and token in ['aquisicao', 'contratacao', 'emergencial', 'dispensa', 'licitacao','compra','comprar','licitatorio', 'contrato']:
                revistas_filtradas[gazette][page] = topics_[page]
                break

In [26]:
for i in range(0,len(revistas_filtradas)):
    revistas_filtradas[i] = list(revistas_filtradas[i].keys())

### resultado da filtragem.

In [3]:
revistas_filtradas = {0: [3, 5],
 1: [1, 2, 4, 11, 12, 13, 14, 16, 17, 18, 19],
 2: [1, 3, 8, 10, 36, 37, 38, 39, 40, 44, 45, 47, 48, 49, 113],
 3: [1, 7, 11, 12, 13, 14, 15],
 4: [1, 2, 4, 5, 6, 7, 8, 9, 10],
 5: [1, 5, 6, 7, 8, 11, 14, 15, 16],
 6: [],
 7: [1, 8, 9, 10, 11, 12, 13],
 8: [2, 11],
 9: [1, 9, 10, 11, 13, 14, 15, 17, 19],
 10: [1, 4, 5, 6, 7, 8, 11, 12],
 11: [1, 10, 11, 12, 13, 15, 16, 17, 18, 19],
 12: [1, 3, 4, 5, 6, 7, 8, 9, 10],
 13: [],
 14: [1, 7, 8, 9, 12, 13, 14, 18],
 15: [1, 5, 6, 7, 8, 9, 12, 13, 14],
 16: [1],
 17: [1, 2, 3, 6, 7, 11, 12, 13, 14, 15, 16, 17],
 18: [1, 2],
 19: [1, 6, 7, 12, 13, 14],
 20: [1, 4, 5, 6, 7, 8, 9, 10],
 21: [2, 3],
 22: [1, 4, 5, 6, 7, 8, 9, 10, 12, 13],
 23: [1, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53],
 24: [1, 38, 39, 40, 50, 51, 55],
 25: [1, 2],
 26: [],
 27: [1, 5, 6, 8, 9, 11, 12, 13, 14, 16, 18],
 28: [1, 7, 9, 12, 13, 14, 16, 17],
 29: [1, 5, 7, 11, 17, 18, 19, 20, 22, 24],
 30: [1, 7, 8, 12, 15, 16, 18],
 31: [1, 2, 6, 7, 8, 9, 10, 11, 12, 13, 14],
 32: [],
 33: [1, 3, 4, 7, 8, 9, 10],
 34: [1, 7, 8, 9, 11, 14, 15, 17, 18, 19, 21],
 35: [1, 8, 19, 20, 23, 25, 26, 27, 28, 29, 31, 33, 34, 35, 36],
 36: [],
 37: [1, 6, 8, 10, 11, 12, 13, 14, 15, 16],
 38: [],
 39: [1, 6, 7, 8, 13, 14, 15, 17],
 40: [1, 3, 5, 8, 9, 10, 11, 12, 13, 17, 18, 19],
 41: [2],
 42: [1, 13, 16, 17, 18, 19, 20, 26, 27, 32],
 43: [1, 4, 8, 9, 10, 13, 14, 15, 16, 17],
 44: [1, 7, 37, 38, 39, 40, 41],
 45: [1, 6, 7, 8, 9, 10, 11, 12],
 46: [],
 47: [1, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21],
 48: [1, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 23],
 49: [],
 50: [1, 4, 8, 9, 10, 11, 12, 13, 14],
 51: [1, 7, 8, 9, 10, 11, 12, 13, 15, 16],
 52: [],
 53: [],
 54: [1, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25],
 55: [1, 2, 3, 4, 5, 6, 7, 8, 9],
 56: [],
 57: [1, 3, 31, 32, 33],
 58: [1, 4, 5, 6, 7, 8, 9],
 59: [],
 60: [1, 13, 15, 16, 17, 18, 20, 21, 22, 23],
 61: [1, 5, 6, 7, 8, 10, 11, 12],
 62: [5],
 63: [1, 7, 12, 13, 14, 18, 19, 21, 22, 23],
 64: [1, 34, 35, 38, 40, 41],
 65: [],
 66: [1, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48],
 67: [1, 10, 12, 13, 14, 16, 17, 18],
 68: [2],
 69: [1, 5, 6, 7, 8, 10, 11],
 70: [],
 71: [1, 5, 8, 10, 11, 14, 15, 19],
 72: [],
 73: [1, 9, 11, 12, 13, 14, 16, 17, 21],
 74: [1, 3, 7, 13, 14, 16, 17, 18, 19, 20],
 75: [],
 76: [1, 7, 8, 9, 10],
 77: [],
 78: [1, 2, 3, 6, 7, 8, 9, 13],
 79: [],
 80: [1, 3, 22, 23, 24, 25, 26, 27, 28, 29],
 81: [1, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18],
 82: [1, 8, 12, 13, 15, 16, 20, 24, 25, 26],
 83: [],
 84: [1, 2],
 85: [1, 5, 6, 7, 8, 10, 11, 12, 13],
 86: [1],
 87: [1, 8, 9, 10, 12, 13, 14, 15],
 88: [],
 89: [],
 90: [1, 5, 7, 9, 10, 11],
 91: [1, 2, 3, 7, 8, 9, 12, 13],
 92: [1, 6, 7, 8, 9, 10, 11, 13, 14, 15],
 93: [],
 94: [1, 4, 5, 8, 9, 10, 11, 12, 13],
 95: [],
 96: [1, 6, 7, 8, 9, 10, 12, 14, 17, 18],
 97: [1, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17],
 98: [1, 4, 5, 6, 10, 12],
 99: [1, 2],
 100: [1, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17],
 101: [1, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23],
 102: [],
 103: [1, 2],
 104: [1, 5, 6, 8, 9, 10, 11, 17],
 105: [1, 2, 3, 4, 5, 8, 9, 10, 15],
 106: [],
 107: [1, 3, 9, 21, 22, 23, 24, 25, 26, 27],
 108: [1, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 109: [],
 110: [1, 15, 16, 17, 18, 19],
 111: [1, 3, 5, 6, 9, 10, 13],
 112: [1, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19],
 113: [],
 114: [1, 6, 7, 9, 10, 11, 12, 13, 17],
 115: [1, 7, 9, 10, 11, 12, 17],
 116: [1, 8, 9, 10, 11, 12, 13, 14],
 117: [1, 2],
 118: [],
 119: [1, 8, 11, 12, 14, 18],
 120: [],
 121: [1, 5, 6, 7, 8, 12, 13, 14, 15, 16, 17, 18, 19],
 122: [1, 12, 13, 25, 26, 27, 28],
 123: [2],
 124: [1, 4, 5, 9, 10, 11, 13, 14, 15, 16, 18],
 125: [],
 126: [1, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
 127: [1, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 20],
 128: [1, 3, 5, 6, 11, 12, 13, 14, 15, 16, 17, 21],
 129: [1, 3, 4, 5, 6, 11, 15, 17, 18, 20],
 130: [1, 5, 6, 8, 9, 10, 11],
 131: [1, 3, 4, 10, 12, 13, 14, 15, 17, 18, 19, 22, 24],
 132: [1, 6, 8, 9, 10, 12, 13, 14, 15],
 133: [1, 7, 8, 9, 13, 16, 18],
 134: [1, 8, 9, 10, 13, 25],
 135: [],
 136: [],
 137: [1, 3, 4, 7, 10, 12, 13, 14, 15, 16],
 138: [1, 6, 7, 8, 9, 10, 11, 12, 13, 80, 106],
 139: [1, 2],
 140: [1, 5, 6, 7, 15, 16, 17, 18, 21, 22],
 141: [2],
 142: [],
 143: [1, 12, 13, 15, 17, 19, 20, 21, 22],
 144: [1, 2, 6, 7, 8, 11, 13, 14, 15, 16, 17, 18, 19, 22],
 145: [],
 146: [],
 147: [1, 11, 12, 13, 15, 16],
 148: [],
 149: [1, 12, 13, 14, 15, 16, 18, 19, 20, 21, 23, 28],
 150: [1, 7, 8, 9, 11, 13, 14],
 151: [],
 152: [1, 7, 8, 9, 10, 15, 16, 21, 22],
 153: [1, 7, 8, 11],
 154: [1, 7, 8, 22, 23, 25, 26, 28, 29],
 155: [1, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
 156: [],
 157: [1, 7, 8, 9, 10, 14, 15, 16, 17, 18, 19, 21],
 158: [1, 6, 13, 14, 29, 30, 31, 35, 40],
 159: [1, 5, 9, 10, 11, 17, 18, 19, 21, 22],
 160: [1, 9, 10, 13, 14, 15, 16, 17, 18, 19, 20, 21, 24, 25, 26, 27],
 161: [],
 162: [1, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17],
 163: [],
 164: [1, 7, 8, 9, 12, 13, 14, 15, 16],
 165: [1, 9, 10, 14, 15, 16, 17, 21],
 166: [1, 33, 34, 38, 39, 40, 41],
 167: [1, 8, 9, 11, 21, 22, 23],
 168: [],
 169: [1, 3, 4, 7, 8, 12, 13, 14, 15],
 170: [1, 9, 10, 12, 13, 15, 16, 17],
 171: [1, 3, 4, 5, 6, 7, 12, 13, 15, 16, 17, 18, 20],
 172: [1, 6, 8, 17, 19, 20, 21, 24, 32, 37, 42, 43, 44, 51, 52, 53],
 173: [3, 4],
 174: [1, 5, 8, 13, 16, 17, 18, 19, 20, 21, 22, 23, 24],
 175: [],
 176: [1, 10, 11, 13, 14, 16],
 177: [1, 29, 30, 31, 32, 34, 39, 40],
 178: [1, 6, 9, 10, 11, 13, 14],
 179: [1, 10, 12, 16, 17, 18, 20, 21, 22],
 180: [],
 181: [1, 47, 48, 52, 53, 54, 55, 56],
 182: [1, 3, 10, 11, 12, 14, 15, 16, 17],
 183: [1, 39, 40, 41, 42, 43, 44, 46, 47, 52],
 184: [1, 3, 5, 6, 9, 10, 12, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55],
 185: [1, 15, 16, 17, 18, 19, 20, 22],
 186: [1, 4, 11, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 28],
 187: [1, 2, 3, 4, 5],
 188: [1, 8, 9, 10, 11, 14, 15, 16, 17, 20, 22, 25, 26, 28, 29, 32, 34, 36, 39],
 189: [],
 190: [1, 7, 42, 45, 49],
 191: [],
 192: [1, 5, 6, 33, 34, 35, 38, 39, 44],
 193: [1, 4, 5, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 20],
 194: [1, 7, 8, 9, 10, 22, 23, 24, 25, 26, 29],
 195: [1, 2],
 196: [1, 5, 7, 8, 17, 24, 25, 26, 27, 28, 31],
 197: [1, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 21, 22, 23, 24, 25, 26, 28, 29, 30, 33, 34],
 198: [1, 6, 13, 14, 15, 17, 22],
 199: [1, 6, 7, 9, 10, 11, 12, 13, 14],
 200: [3],
 201: [1, 12, 13, 14, 15, 17, 18, 19, 21, 22, 23],
 202: [1, 8, 9, 10, 11, 12, 13, 17],
 203: [],
 204: [1, 17, 18, 19, 20, 21, 22, 23],
 205: [1, 6, 10, 11, 12, 13, 14, 15, 16, 17, 33],
 206: [2],
 207: [1, 6, 9, 10, 13, 14, 16, 17, 18, 19, 23],
 208: [1, 9, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
 209: [1, 3, 4, 5, 6, 7, 11, 12, 14, 15, 16, 17, 18],
 210: [1, 10, 13, 14, 15, 17, 18, 19, 20, 22, 23],
 211: [1, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 26, 27],
 212: [1, 5, 20, 21, 22, 25, 26, 31, 32, 33, 34, 36],
 213: [1, 6, 11, 12, 18, 19, 20, 21, 22],
 214: [1, 6, 10, 23, 24, 25, 28, 30, 31, 33],
 215: [],
 216: [],
 217: [1, 8, 9, 10, 11, 12, 13, 14, 17],
 218: [2, 3],
 219: [1, 10, 15, 16, 18, 21, 22, 25],
 220: [1, 5, 6, 13, 14, 15, 16, 17, 18, 19, 20, 21],
 221: [],
 222: [1, 7, 8, 9, 10, 12, 13, 17, 21, 22, 23, 24, 27],
 223: [1, 3, 14, 15, 16, 18, 20, 21, 22, 23, 24, 25],
 224: [1, 18, 19, 23, 25, 26, 27, 28],
 225: [1, 8, 9, 10, 11, 12, 13, 14, 15, 16],
 226: [1, 4, 8, 9, 10, 12, 13, 14, 15, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35],
 227: [1, 7, 11, 12, 13, 14, 16, 17, 18, 21, 22, 23, 41],
 228: [],
 229: [1, 10, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25],
 230: [],
 231: [1, 7, 9, 10, 11, 14, 15, 16, 23],
 232: [1, 11, 12, 17, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32],
 233: [1, 10, 12, 14, 15, 16, 17, 19, 20, 27, 28, 32],
 234: [1, 8, 11, 12, 13, 14, 15, 17, 18, 19],
 235: [1, 8, 12, 13, 14, 15, 19, 20, 24],
 236: [1, 11, 13, 14, 15, 16],
 237: [1, 6, 14, 15, 16, 19, 21],
 238: [1, 3, 7, 8, 9, 10, 20, 21, 25, 29, 30],
 239: [1, 24, 25, 27, 29, 30, 32],
 240: [2],
 241: [1, 8, 11, 14, 15, 16, 17, 19, 22, 23],
 242: [1, 7, 8, 14, 19, 20, 21, 23, 24, 25, 34, 36],
 243: [1, 6, 7, 10, 22, 23, 24, 25, 26, 29, 31],
 244: [],
 245: [1, 10, 14, 15, 16, 17, 18, 19, 20, 26],
 246: [1, 6, 12, 13, 15, 16, 17, 18, 19, 25, 26, 28, 30, 31],
 247: [1, 6, 8, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 25],
 248: [1, 30, 31, 32, 39, 40, 41, 42, 48],
 249: [1, 5, 11, 12, 13, 14, 16, 17, 18, 19],
 250: [],
 251: [1, 4, 5, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 30],
 252: [1, 11, 14, 16, 17, 19, 20, 21, 22, 23],
 253: [],
 254: [],
 255: [1, 6, 11, 12, 13, 14, 16, 17, 21],
 256: [],
 257: [1, 8, 12, 14, 21, 22, 23, 24, 25, 27],
 258: [1, 11, 12, 13, 18, 19, 20, 22, 23],
 259: [3],
 260: [1, 25, 26, 27, 28, 29, 31, 32, 33, 34],
 261: [],
 262: [1, 7, 10, 11, 12, 13, 14, 15, 18, 19, 20, 21],
 263: [],
 264: [],
 265: [1, 5, 14, 15, 16, 19, 21, 22, 23, 24],
 266: [1, 9, 10, 11, 12, 13, 14, 15, 16, 17],
 267: [],
 268: [1, 3, 11, 20, 21, 22, 23, 24, 25, 26, 27],
 269: [],
 270: [1, 4, 7, 8, 14, 15, 16, 18, 19, 28],
 271: [1, 5, 6, 7, 9, 13, 14, 15, 16, 17, 18],
 272: [1, 2],
 273: [1, 6, 18, 21, 22, 23, 24, 28, 29],
 274: [1, 3, 23, 24, 25, 35],
 275: [1, 7, 8, 14, 15, 16, 17, 25, 110, 111, 112, 113, 114, 115, 116, 118],
 276: [1, 5, 86, 87, 90, 93, 101, 105, 106, 107, 109, 110, 111],
 277: [],
 278: [],
 279: [],
 280: [],
 281: [],
 282: [1, 70, 71, 81],
 283: [1, 6, 9, 10, 12, 13, 14, 19, 20],
 284: [1, 3, 8, 9, 10, 11, 15],
 285: [1, 36, 37, 40, 41, 42, 44, 48, 49, 54, 55],
 286: [],
 287: [1, 7, 9, 11],
 288: [1, 7, 9, 15, 16, 17, 19],
 289: [1, 20, 21, 22, 23, 32],
 290: [1, 4, 5, 6, 9],
 291: [1, 2, 3, 15, 16, 18, 19, 20],
 292: [1, 2, 5, 6, 7, 14, 15, 17],
 293: [],
 294: [1, 10, 11, 12],
 295: [1, 4, 7, 11, 12],
 296: [1, 8, 12, 13, 15, 16, 17, 18],
 297: [1, 3, 5, 6, 7, 8, 9, 12, 13],
 298: [1, 5, 6, 9, 10, 12, 13, 14],
 299: [1, 3, 4, 8, 10, 18, 19, 25, 31],
 300: [1, 10, 11, 12, 13, 14, 16, 22, 23, 24],
 301: [1, 7, 8, 9, 10, 11, 12, 14, 15, 21, 22],
 302: [1, 7, 8, 9, 10, 13, 14, 15, 16, 17, 18],
 303: [1, 12, 13, 15, 18, 20, 21],
 304: [1, 10, 11, 12, 14, 22, 60, 61],
 305: [],
 306: [1, 6, 15, 16, 17, 18, 20, 21, 30],
 307: [1, 7, 13, 14, 15, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30],
 308: [1, 8, 10, 12, 13, 14, 15, 16, 17, 18, 24, 25, 26, 28, 30, 31, 33, 34, 37],
 309: [1, 6, 7, 8, 9, 10, 12, 19, 21, 22, 23, 24, 26, 27],
 310: [1, 11, 12, 17, 20, 24],
 311: [1, 8, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20],
 312: [],
 313: [1, 6, 7, 8, 9, 10, 16, 17, 19, 21, 22, 23],
 314: [1, 8, 9, 12, 17, 18, 25, 26, 27, 28, 33, 34],
 315: [1, 8, 9, 15, 16, 17, 25, 26, 27],
 316: [],
 317: [1, 11, 13, 14, 15, 23, 24, 25],
 318: [1, 6, 11, 13, 14, 17, 18, 19, 20, 24, 26, 27],
 319: [1, 10, 11, 14, 15, 16, 17, 18, 21, 22],
 320: [1, 6, 10, 11, 13, 14, 16, 17, 18, 19, 20, 21, 22, 61],
 321: [],
 322: [1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 29, 30, 32, 34, 35],
 323: [],
 324: [1, 4, 10, 11, 13, 14],
 325: [1, 7, 8, 9, 11, 12, 13, 14, 15, 16],
 326: [],
 327: [2],
 328: [1, 10, 11, 13, 14, 15, 16, 17, 18],
 329: [1, 9, 10, 13, 14, 16, 19],
 330: [1, 7, 8, 12, 13],
 331: [],
 332: [],
 333: [2],
 334: [1, 3, 4, 6, 8, 9, 10, 11, 12, 16, 17, 18, 19],
 335: [1],
 336: [1, 7, 8, 9, 12],
 337: [1, 14, 15, 16, 17, 18, 20],
 338: [],
 339: [1, 12, 14, 15, 16, 19, 20, 21],
 340: [1, 10, 11, 13, 14, 15, 16, 17, 20, 21, 24],
 341: [2],
 342: [1, 12],
 343: [1, 5, 6, 7, 8, 9, 10, 11, 12],
 344: [1, 3, 6, 7, 8, 13, 15, 16, 17],
 345: [1, 13, 14, 15, 16, 18, 20, 21, 24, 25],
 346: [1, 4, 7, 8, 9, 10, 15, 16, 17, 18],
 347: [1],
 348: [1, 9, 10, 16, 17, 18],
 349: [1, 2, 4, 5, 6, 11, 12],
 350: [1, 8, 9, 11, 12, 15, 16],
 351: [1, 5, 22, 23, 25, 26, 27, 30, 31, 32],
 352: [1, 23, 25, 36, 44, 45, 50],
 353: [1, 5, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27],
 354: [2],
 355: [1, 4, 8, 10, 11, 13],
 356: [],
 357: [1, 9, 14, 15, 16, 17, 19, 20, 22, 23, 24],
 358: [1, 7, 8, 9, 12, 14, 19, 20],
 359: [1],
 360: [1, 2, 3, 15, 16, 17],
 361: [1, 10, 11, 12, 14, 15, 16, 17, 18, 22, 23, 24, 25],
 362: [1, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22],
 363: [1],
 364: [1, 2],
 365: [1, 3, 7, 8, 9, 10, 11, 12, 13, 14, 16],
 366: [1, 7, 8, 12, 18, 19, 20],
 367: [1],
 368: [1, 9, 14, 15, 19, 20],
 369: [],
 370: [3, 13, 14],
 371: [1, 8, 12, 14, 15, 16, 17, 18, 19, 20],
 372: [1, 5, 6, 7, 8, 10, 11, 13, 14],
 373: [5],
 374: [1, 5, 16, 17, 18, 19, 20, 21, 22, 23, 34]}

In [3]:
revistas_filtradas = {0: [4, 5],
 1: [1, 2, 3, 4, 5, 6, 12, 13, 16, 19],
 2: [1, 2, 3, 4, 5, 6, 8, 10, 36, 37, 38, 39, 40, 43, 44, 48],
 3: [1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 14],
 4: [1, 2, 3, 4, 5, 8, 10],
 5: [1, 2, 3, 4, 5, 6, 11, 12, 13, 16, 19],
 6: [1, 2],
 7: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 8: [1],
 9: [1, 2, 3, 4, 5, 6, 8, 9, 12, 13, 17, 18],
 10: [1, 2, 3, 4, 5, 7, 9, 10, 12],
 11: [1, 2, 9, 10, 13, 14, 15, 19],
 12: [1, 2, 4, 7, 10],
 13: [1],
 14: [1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 13, 15, 16, 17, 18],
 15: [1, 2, 3, 4, 5, 8, 9, 10, 11],
 16: [1],
 17: [1, 3, 4, 5, 8, 9, 10, 11, 15, 16],
 18: [1],
 19: [1, 2, 3, 4, 10],
 20: [1, 2, 4, 5, 6, 8],
 21: [1, 2],
 22: [1, 2, 3, 6, 7, 8, 9, 10],
 23: [1, 2, 3, 4, 5, 6, 7, 41, 47, 48],
 24: [1, 2, 3, 5, 6, 9, 10, 11, 38, 40, 47, 48, 49, 51, 55],
 25: [1],
 26: [1],
 27: [1, 2, 3, 4, 5, 6, 8, 9, 10],
 28: [1, 2, 5, 7, 12],
 29: [1, 2, 13, 16, 17, 18, 22],
 30: [1, 2, 3, 4, 5, 6, 7, 8, 12, 13, 14, 15],
 31: [1, 2, 3, 4, 5, 6, 9, 10, 11, 14],
 32: [1],
 33: [1, 2, 3, 8, 10],
 34: [1, 2, 3, 4, 5, 6, 7, 9, 13, 15, 16],
 35: [1, 2, 3, 4, 5, 7, 8, 9, 18, 19, 20, 23, 24, 35],
 36: [1, 2],
 37: [5, 6, 7, 8, 10, 13, 14],
 38: [1],
 39: [1, 2, 3, 4, 5, 7, 8, 14, 15, 16, 17],
 40: [1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 14, 15, 17, 19],
 41: [1, 2],
 42: [1,
  2,
  3,
  5,
  6,
  7,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  23,
  27,
  28,
  29,
  30,
  31,
  32],
 43: [1, 2, 13, 14],
 44: [1, 2, 3, 6, 7, 36, 37],
 45: [1, 2, 3, 5, 6, 8, 10, 12],
 46: [1, 2, 3],
 47: [1, 2, 3, 4, 5, 9, 10, 21],
 48: [1, 2, 3, 4, 5, 6, 8, 11, 12, 13, 14, 15, 16, 19],
 49: [1, 2],
 50: [1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14],
 51: [1, 2, 3, 6, 7, 11, 12, 13],
 52: [1, 2, 3],
 53: [1],
 54: [1, 2, 3, 4, 5, 10, 11, 14, 15, 16, 19, 20, 21, 24],
 55: [1, 2, 6, 7, 8],
 56: [1],
 57: [1, 3, 4, 18, 27, 33],
 58: [1, 2, 3, 5, 9],
 59: [1, 2, 3],
 60: [1, 2, 3, 4, 5, 12, 13, 14, 15, 16, 19, 20],
 61: [1, 2, 3, 5, 6, 9],
 62: [1, 3, 4, 6],
 63: [1, 2, 3, 4, 5, 8, 9, 10, 11, 14, 21, 23],
 64: [1, 2, 33, 34, 42],
 65: [1, 2],
 66: [1, 2, 3, 4, 5, 35, 36, 37, 38, 40, 41, 42, 43, 44],
 67: [1, 2, 3, 4, 9, 10, 11, 12, 17, 18],
 68: [1, 2, 3],
 69: [1, 2, 3, 4, 5, 6, 7, 8, 10, 11],
 70: [1],
 71: [1, 2, 5, 6, 12, 14, 15, 16],
 72: [1],
 73: [1, 2, 3, 7, 8, 9, 10, 11, 14, 16, 17, 18, 21],
 74: [1, 2, 3, 5, 6, 7, 8, 9, 10, 12, 14, 15],
 75: [1],
 76: [1, 2, 3, 4, 5, 6, 7, 9, 10],
 77: [1, 2, 3],
 78: [1, 2, 3, 5, 6, 8, 12],
 79: [1, 2],
 80: [1, 2, 3, 4, 21, 22, 26],
 81: [1, 2, 3, 7, 8, 9, 13, 14],
 82: [1, 2, 3, 4, 8, 9, 10, 12, 13, 14, 15, 17, 20, 27, 28],
 83: [1],
 84: [1],
 85: [1, 2, 3, 4, 5, 6, 8, 10, 11, 13],
 86: [1],
 87: [1, 2, 4, 6, 7, 8, 12, 13],
 88: [1, 2],
 89: [1],
 90: [1, 2, 3, 4, 5, 6, 7, 10],
 91: [2, 3, 4, 5, 6, 9],
 92: [1, 2, 3, 4, 6, 8, 10, 11, 12],
 93: [1, 2],
 94: [1, 2, 3, 4, 5, 7, 10, 11],
 95: [1, 2, 4, 5],
 96: [1, 2, 5, 6, 7, 8, 9, 10, 14],
 97: [1, 2, 8, 9, 10, 11, 12],
 98: [2, 3, 4, 5, 10],
 99: [1, 2, 3],
 100: [1, 2, 3, 4, 5, 7, 12],
 101: [1, 2, 3, 6, 7, 8, 9, 11, 12, 14, 15, 16, 18, 21, 22],
 102: [1],
 103: [1, 2, 3],
 104: [1, 2, 3, 4, 6, 7, 9, 15, 17],
 105: [1, 2, 3, 4, 5, 7, 14],
 106: [1, 2],
 107: [1, 2, 3, 4, 5, 8, 9, 21, 25, 27],
 108: [1, 2, 3, 4, 5, 6, 8, 10, 11, 12],
 109: [1, 2],
 110: [1, 2, 9, 10, 12, 13, 14, 15, 16, 18, 19],
 111: [1, 2, 4, 5, 6, 10],
 112: [1, 2, 8, 10, 11, 12, 13, 14, 15],
 113: [1, 2, 3, 4],
 114: [1, 2, 3, 4, 6, 7, 9, 10, 12, 16],
 115: [1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 13, 15, 16, 17],
 116: [1, 2, 3, 4, 6, 10, 12, 13, 14],
 117: [1, 2, 3],
 118: [1],
 119: [1, 2, 3, 5, 6, 7, 8, 9, 11, 12, 13, 15],
 120: [1],
 121: [1, 2, 3, 8, 11, 12, 13],
 122: [1, 2, 3, 5, 7, 8, 9, 10, 13, 26, 27, 29, 30],
 123: [1],
 124: [1, 2, 3, 5, 6, 8, 9, 13, 16, 17],
 125: [1],
 126: [1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 16],
 127: [1, 2, 3, 5, 6, 7, 12, 13, 14, 16],
 128: [1, 2, 3, 4, 9, 10, 11, 12, 14, 15, 16, 17, 18],
 129: [1, 2, 6, 7, 10, 11, 12, 13, 14, 16, 17, 19, 20],
 130: [1, 2, 3, 5, 7, 11],
 131: [1, 2, 3, 4, 12, 13, 15, 16, 17, 18, 20, 23],
 132: [1, 2, 3, 4, 5, 6, 7, 10, 11, 14],
 133: [1, 2, 3, 4, 7, 13, 14, 17, 18],
 134: [1, 2, 3, 7, 8, 9, 10, 12, 13, 24, 25],
 135: [1, 2, 3, 4],
 136: [1, 2, 3],
 137: [1, 2, 3, 4, 5, 8, 12, 13, 14, 15],
 138: [1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 106],
 139: [],
 140: [1, 2, 3, 4, 15],
 141: [1, 2],
 142: [],
 143: [1, 2, 3, 4, 6, 7, 9, 10, 12, 14, 18, 23, 25, 26],
 144: [1, 4, 9, 10, 11, 12, 15, 16, 17, 18],
 145: [1, 2],
 146: [1, 2],
 147: [1, 2, 3, 4, 5, 6, 7, 10, 11, 15, 16, 18],
 148: [1, 2],
 149: [1, 2, 3, 4, 6, 10, 11, 12, 13, 14, 17, 19, 21],
 150: [1, 2, 3, 4, 5, 6, 9, 10, 13, 14],
 151: [1],
 152: [1, 2, 3, 4, 5, 6, 8, 10, 17, 18],
 153: [1, 2, 6, 7, 8, 9, 10, 12, 15],
 154: [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 19, 20, 22, 23, 24, 28, 29],
 155: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 20, 21],
 156: [1, 2, 3],
 157: [1, 2, 3, 7, 8, 10, 14, 15, 16, 17, 18, 19, 20, 21],
 158: [1,
  2,
  3,
  4,
  5,
  7,
  17,
  18,
  19,
  20,
  21,
  23,
  28,
  29,
  31,
  32,
  34,
  35,
  36,
  39,
  42],
 159: [1, 2, 3, 4, 5, 7, 8, 9, 10, 20, 22],
 160: [1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 32],
 161: [1, 2, 3, 4],
 162: [1, 2, 3, 6, 9, 10, 11, 15, 16, 17, 18],
 163: [1, 2, 3, 4],
 164: [1, 2, 3, 4, 6, 8, 12, 13, 16],
 165: [1, 2, 3, 4, 5, 6, 8, 9, 12, 13, 17, 21],
 166: [1, 2, 3, 6, 7, 8, 9, 10, 13, 15, 30, 32, 33, 34, 38, 39],
 167: [1, 2, 4, 6, 7, 8, 10, 22, 23],
 168: [1, 3, 4, 5],
 169: [1, 2, 3, 5, 8, 9, 11, 13, 16],
 170: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 15, 22],
 171: [1, 2, 3, 9, 12, 13, 15, 16, 20],
 172: [1,
  2,
  3,
  4,
  5,
  7,
  20,
  30,
  31,
  32,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  43,
  47,
  52],
 173: [1, 2, 3, 4, 5, 6, 7],
 174: [1, 2, 4, 5, 6, 7, 8, 9, 14, 16, 17, 18, 19, 20, 21, 22, 24, 25],
 175: [1, 2],
 176: [1, 2, 3, 4, 5, 8, 9, 10, 12, 13, 14, 15],
 177: [1, 2, 4, 5, 14, 28, 29, 33, 39],
 178: [1, 2, 3, 4, 5, 6, 7, 8, 9],
 179: [1, 2, 3, 4, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 22],
 180: [1, 2],
 181: [1, 2, 3, 8, 9, 10, 46, 47, 50, 51, 54, 56],
 182: [1, 4, 5, 7, 8, 9, 13, 14, 15, 16],
 183: [1, 2, 3, 4, 5, 38, 39, 42, 43, 44, 47],
 184: [1, 2, 4, 8, 10, 11, 43, 45, 48, 49, 50, 51, 52, 55],
 185: [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 21],
 186: [1,
  2,
  3,
  4,
  6,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  19,
  20,
  21,
  23,
  24,
  26],
 187: [1, 2, 4, 5, 6, 7],
 188: [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  12,
  14,
  16,
  17,
  19,
  20,
  21,
  22,
  23,
  25,
  27,
  28,
  29,
  31,
  32,
  33,
  34,
  35,
  37,
  38,
  39,
  40],
 189: [1],
 190: [1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 26, 41, 42, 43, 45, 46, 48],
 191: [1],
 192: [1, 2, 3, 4, 6, 7, 8, 31, 33, 34, 40, 41, 44, 45, 46],
 193: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 17, 19, 20],
 194: [1, 2, 3, 4, 5, 7, 8, 9, 10, 24, 27, 28, 29, 30],
 195: [2, 3, 4, 5],
 196: [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 16, 18, 19, 20, 31, 32],
 197: [1, 2, 3, 4, 5, 6, 12, 16, 22, 24, 25, 26],
 198: [1, 2, 3, 4, 6, 7, 12, 14, 15, 18, 25, 26],
 199: [1, 2, 3, 4, 5, 6, 7, 9, 12],
 200: [2, 3],
 201: [1, 2, 3, 4, 5, 9, 10, 11, 12, 16, 17, 22],
 202: [1, 2, 3, 5, 6, 7, 8, 10, 11, 12, 13, 17],
 203: [1],
 204: [1, 2, 7, 9, 10, 14, 15, 18, 19, 24],
 205: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 33],
 206: [1, 2, 3],
 207: [1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 16, 20],
 208: [1, 2, 3, 4, 6, 8, 9, 10, 11, 12, 13, 16, 23, 33, 35],
 209: [1, 2, 9, 10, 12, 17, 18],
 210: [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24],
 211: [1, 2, 3, 4, 5, 6, 9, 12, 13, 20, 22],
 212: [1,
  2,
  3,
  4,
  5,
  8,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  25,
  28,
  29,
  30,
  31,
  34,
  37,
  46,
  48],
 213: [1, 2, 3, 6, 9, 19],
 214: [1, 2, 3, 4, 5, 9, 15, 16, 17, 20, 22, 23, 25, 26],
 215: [1],
 216: [1],
 217: [1, 2, 3, 4, 6, 7, 8, 9, 10, 11],
 218: [1, 2, 3],
 219: [1, 2, 3, 4, 5, 6, 7, 15, 16, 18, 19, 25],
 220: [1, 2, 3, 4, 6, 7, 8, 9, 12, 13, 14, 15, 16],
 221: [1, 2],
 222: [1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 14, 15, 16, 21, 25, 26, 27],
 223: [1, 2, 3, 4, 5, 7, 8, 9, 13, 14, 19, 20],
 224: [1, 2, 3, 4, 15, 17, 18, 24, 25],
 225: [1, 2, 3, 4, 8, 9, 11, 15],
 226: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 16, 24, 35],
 227: [1, 2, 3, 4, 6, 7, 11, 12, 24, 40, 41],
 228: [1],
 229: [1, 2, 3, 6, 7, 10, 12, 13, 14, 15, 17, 19, 20, 23],
 230: [1, 2],
 231: [1, 2, 3, 4, 6, 7, 8, 12, 16],
 232: [1, 2, 3, 13, 14, 15, 16, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 32],
 233: [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 14, 15, 16, 20, 27, 28, 31, 32],
 234: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 20],
 235: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 18, 19, 21],
 236: [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 14, 15],
 237: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 20, 22],
 238: [1, 2, 6, 10, 11, 12, 13, 14, 15, 16, 17, 22, 23, 24, 29, 32, 35],
 239: [1, 2, 3, 4, 5, 6, 8, 9, 24, 26, 28, 29, 32],
 240: [1, 2],
 241: [1, 2, 3, 4, 6, 7, 8, 10, 11, 12, 13, 15, 16, 17, 21],
 242: [1, 2, 3, 4, 5, 6, 9, 10, 12, 13, 14, 18, 19, 21, 22, 23, 34, 35, 36],
 243: [1,
  2,
  3,
  4,
  5,
  6,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  22,
  25,
  26,
  28,
  29,
  30,
  31,
  32,
  35,
  38],
 244: [1],
 245: [1, 2, 3, 4, 5, 6, 8, 10, 14, 16, 20],
 246: [1, 2, 3, 4, 8, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 26, 30, 31],
 247: [1, 2, 3, 4, 7, 8, 9, 10, 11, 13, 14, 15, 17, 20, 22, 25],
 248: [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 25, 26, 29, 30, 31, 32, 34, 43, 45, 47],
 249: [1, 2, 3, 6, 7, 9, 10, 14, 15, 19, 20],
 250: [1, 2],
 251: [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  14,
  15,
  16,
  19,
  20,
  21,
  22,
  23,
  24,
  30],
 252: [1, 2, 3, 4, 5, 6, 7, 11, 12, 13, 14, 15, 17, 18, 22],
 253: [1, 2, 3],
 254: [1, 2, 3],
 255: [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 16, 19],
 256: [1, 2, 3, 4, 5, 6, 7, 8, 9, 17, 18, 19, 20, 21, 23],
 257: [1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 15, 17, 18, 21, 23, 29, 32],
 258: [1, 2, 4, 5, 8, 10, 12, 23],
 259: [1, 2, 3],
 260: [1,
  2,
  3,
  4,
  5,
  9,
  10,
  15,
  16,
  17,
  18,
  19,
  22,
  23,
  25,
  27,
  28,
  30,
  33,
  34],
 261: [1, 2, 3, 4],
 262: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14],
 263: [1, 2, 3, 4, 5],
 264: [1, 2, 3, 4, 5, 6],
 265: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20],
 266: [1, 2, 3, 4, 6, 7, 8, 9, 10],
 267: [1],
 268: [1,
  2,
  3,
  4,
  5,
  8,
  10,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24],
 269: [1],
 270: [1, 2, 3, 4, 5, 10, 11, 12, 14, 18, 19, 20, 21, 28],
 271: [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 16],
 272: [2, 7, 8],
 273: [2, 4, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 29],
 274: [3, 4, 5, 6, 7, 8, 9, 22, 23, 24, 25, 28, 29, 30, 31],
 275: [1,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  21,
  22,
  23,
  25,
  26,
  30,
  31,
  32,
  43,
  52,
  64,
  85,
  94,
  110,
  112,
  115,
  118],
 276: [1,
  3,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  99,
  100,
  101,
  102,
  103,
  105,
  111,
  112,
  113,
  114,
  115,
  116,
  118,
  119,
  120,
  121,
  122],
 277: [1],
 278: [1],
 279: [1],
 280: [1],
 281: [1],
 282: [1, 2, 3, 14, 23, 35, 49, 69, 75],
 283: [1, 2, 5, 8, 9, 10, 13, 14, 16],
 284: [1, 2, 7, 8, 9, 10, 11, 14],
 285: [1, 2, 35, 37, 38, 45, 48, 49, 50, 51, 52, 53, 55, 58],
 286: [1, 2, 3, 4, 5],
 287: [1, 2, 4, 5, 6, 7, 10, 11],
 288: [1, 2, 12, 13, 14, 15, 17],
 289: [1, 2, 3, 4, 10, 11, 17, 18, 19, 20, 21, 22, 23, 24, 26, 32, 33],
 290: [1, 2, 3, 4, 5, 9],
 291: [1, 2, 3, 15, 16, 17],
 292: [1, 2, 3, 4, 5, 6, 7, 16],
 293: [1],
 294: [1, 2, 3, 6, 7, 8, 9, 10, 12],
 295: [1, 2, 4, 6, 7, 8, 10],
 296: [1, 2, 3, 7, 8, 9, 10, 12, 16, 18],
 297: [1, 2, 3, 4, 5, 6, 7, 8, 10],
 298: [1, 2, 3, 4, 5, 6, 7, 8, 9, 13, 14],
 299: [1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 14, 15, 16, 17, 19, 20, 25, 31],
 300: [1, 2, 3, 9, 10, 11, 17],
 301: [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 17],
 302: [1, 2, 3, 4, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16],
 303: [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14],
 304: [1, 2, 3, 4, 10, 11, 27, 60],
 305: [1],
 306: [1, 2, 3, 4, 5, 6, 14, 15, 17, 19, 20, 22, 24],
 307: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 20, 21, 22, 23, 24, 25, 26, 29],
 308: [1, 2, 3, 4, 5, 6, 10, 19, 20, 23, 24, 27, 28, 29, 30, 31, 34],
 309: [1, 2, 3, 4, 5, 6, 8, 20, 21, 25, 26],
 310: [1, 2, 3, 4, 9, 10, 11, 12, 17, 18, 21],
 311: [1, 2, 3, 4, 5, 6, 7, 9, 10, 13, 14, 15, 16, 17, 19],
 312: [1, 2],
 313: [1, 2, 3, 4, 6, 8, 9, 16, 17, 18, 19],
 314: [1, 2, 3, 4, 5, 6, 11, 12, 13, 14, 16, 25],
 315: [1, 2, 3, 4, 5, 6, 7, 8, 13, 14, 15, 17, 18, 19, 20, 21, 24, 25, 26, 27],
 316: [1],
 317: [1, 2, 3, 10, 11, 12, 13, 14, 16, 17, 18, 19, 27, 28],
 318: [1, 2, 3, 4, 5, 6, 7, 13, 16, 17, 19, 20, 27, 28],
 319: [1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18],
 320: [1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 15, 16, 17, 18, 20, 25],
 321: [1, 2],
 322: [1, 2, 3, 13, 14, 15, 16, 21, 28, 31, 32],
 323: [1, 2, 4],
 324: [1, 2, 3, 4, 7, 8, 9, 10, 11, 13, 14, 15, 19],
 325: [1, 2, 3, 4, 6, 7, 8, 9, 10, 11],
 326: [1],
 327: [1, 2],
 328: [1, 2, 3, 5, 6, 7, 8, 9, 13, 14],
 329: [1, 2, 3, 4, 5, 7, 8, 12, 13, 14, 15, 16],
 330: [1, 2, 3, 4, 5, 7, 10, 12],
 331: [1, 2],
 332: [1, 2],
 333: [1, 2],
 334: [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 17],
 335: [1, 2, 3],
 336: [1, 2, 3, 4, 5, 6, 8, 9, 10, 11],
 337: [1, 2, 3, 14, 16, 17, 18],
 338: [1],
 339: [1, 2, 11, 12, 13, 14, 21],
 340: [1, 2, 3, 4, 9, 10, 11, 12, 13, 15, 17, 18, 20],
 341: [1, 2],
 342: [1],
 343: [1, 2, 3, 5, 7, 8, 9, 11, 12],
 344: [1, 2, 3, 4, 5, 6, 8, 9, 12],
 345: [1, 2, 3, 4, 5, 6, 7, 8, 13, 14, 15, 16, 20, 21, 22, 24],
 346: [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 16],
 347: [1, 2],
 348: [1, 2, 3, 4, 5, 6, 7, 8, 9, 13, 15],
 349: [1, 2, 3, 4, 6, 7, 8, 10, 11, 12],
 350: [1, 2, 4, 6, 7, 8, 9, 11, 12, 13],
 351: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 22, 24, 25, 26, 27],
 352: [1, 2, 3, 4, 8, 16, 17, 20, 22, 24, 25, 26, 35, 38, 43, 44, 45, 46],
 353: [1, 2, 3, 4, 5, 18, 19, 20, 21, 22, 23, 25, 26],
 354: [1, 2],
 355: [1, 2, 3, 4, 5, 6, 7, 10, 11],
 356: [1, 2],
 357: [1, 2, 3, 4, 5, 8, 9, 10, 14, 15, 18, 19, 23, 24],
 358: [1, 2, 3, 4, 7, 8, 9, 11, 12, 15],
 359: [],
 360: [1, 9, 10, 15, 16, 17, 18],
 361: [1, 2, 3, 4, 8, 11, 12, 13, 14, 15, 17, 22, 23, 25],
 362: [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 17, 18, 20],
 363: [1],
 364: [1, 2],
 365: [1, 2, 3, 4, 5, 10, 11, 14],
 366: [1, 2, 3, 4, 5, 10, 11, 12, 13, 19, 22, 23, 25, 26],
 367: [],
 368: [1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 15, 22, 23, 24, 25],
 369: [1, 2, 3],
 370: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14],
 371: [1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 17, 20],
 372: [1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 15],
 373: [1, 2, 3, 4],
 374: [1,
  2,
  3,
  4,
  5,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  19,
  20,
  21,
  22,
  34]}

### Extraindo de multiplas gazetas

In [4]:
pp = ProcessGazette(BASE_DIR="gazettes/")
txt_files = [f for f in os.listdir("gazettes/") if f.endswith('.txt')]
all_pages = []
for txt_file in txt_files:
    pages = pp.break_pages(txt_file, "ANO [X|V|I]+ ", save_file=True)
    all_pages.append(pages)

In [5]:
cont = 2 #len(revistas_filtradas)

In [6]:
all_topics = {}
topics_ = {}
ppt = PreProcessText("pt_core_news_lg")
for gazette in range(0,cont):
    all_topics[gazette] = {}
    for page, text in all_pages[gazette].items():
        if page in revistas_filtradas[gazette]:
            extractor = ExtractTopics(model = 'default')
            #extractorGpt = ExtractTopics(model = 'gpt')
            extractorZeroshot = ExtractTopics(model = 'zeroshot')
            extractorMaritalk = ExtractTopics(model = 'maritalk')
            ner = Ner()
            topics_[page] = {'text': text, 'topics': []}
            tokens_ = ppt.process_text(topics_[page]['text'])
            if len(tokens_) > 5 and page:
                topics_[page]['tokens'] = tokens_
                topics_[page]['topics'], topics_[page]['topics_info'] = extractor.extract_topics(topics_[page]['tokens'])
                #topics_[page]['topicsGpt'], topics_[page]['topics_infoGpt'] = extractorGpt.extract_topics(topics_[page]['tokens'])
                topics_[page]['topicsZeroshot'], topics_[page]['topics_infoZeroshot'] = extractorZeroshot.extract_topics(topics_[page]['tokens'])
                topics_[page]['topicsMaritalk'], topics_[page]['topics_infoMaritalk'] = extractorMaritalk.extract_topics(topics_[page]['tokens'])
                
                topics_[page]['Ner'] = ner.extract_entities_tokens(topics_[page]['tokens'])
                topics_[page]['Ner_text'] = ner.extract_entities_text(topics_[page]['text'])
                topics_[page]['Ner_mari_txt'] = ner.extract_entities_maritalk(topics_[page]['text'])
                topics_[page]['Ner_mari'] = ner.extract_entities_maritalk(topics_[page]['tokens'])
                topics_[page]['similarity'] = extractor.find_topics('Licitação')
                all_topics[gazette][page] = topics_[page]


c:\Users\erick\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### run

In [7]:
gazeta = 0
pg = 4

In [15]:
words = all_topics[gazeta][pg]['text'].split()
new_text = ""
for i, word in enumerate(words):
    new_text += word + " "
    if (i + 1) % 20 == 0:
        new_text += "\n"

print(new_text)

| N º 7.6114 Art. 13. As ações previstas neste Capítulo têm caráter temporário e excepcional, devendo perdurar durante o 
período da pandemia. Parágrafo único. Fica autorizada a prorrogação de ofício do termo de colaboração ou fomento, acordos de cooperação, 
termos de parceria, contratos de repasse, contratos de gestão, convênios e demais ajustes similares com vencimento a partir de 19 
de março de 2020, data de decretação do estado de emergência no Município de Salvador, pelo prazo de duração da 
emergência até 3 (três) meses após a decretação do fim da emergência, nas mesmas condições avençadas, devendo ser observados os 
demais aspectos procedimentais da legislação pertinente ao ajuste prorrogado. Art. 14. Fica autorizado o estabelecimento, mediante Decreto, de regras excepcionais 
em relação à prestação de contas quanto aos ajustes que sofram as modificações previstas no art. 10 desta Lei, desde 
que justificadas ante o cenário de emergência e que estejam de acordo com a legislaçã

In [8]:
all_topics[gazeta][pg]['topics_info'][:5]

,Topic,Count,Name,Representation,Representative_Docs
0,-1,104,-1_acordo_objetivo_compor_finalidade,"[acordo, objetivo, compor, finalidade, convida...","[acordo, compor, acordo]"
1,0,23,0_urbanismo___,"[urbanismo, , , , , , , , , ]","[municipal, urbanismo, urbanismo]"
2,1,19,1_entidade_instituicao_gabinete_desenvolvimento,"[entidade, instituicao, gabinete, desenvolvime...","[entidade, entidade, entidade]"
3,2,18,2_secretaria_geral_secretario_,"[secretaria, geral, secretario, , , , , , , ]","[secretario-geral, Secretaria, Secretaria]"
4,3,18,3_situacao_indireto_calamidade_conforme,"[situacao, indireto, calamidade, conforme, sim...","[indireto, indireto, situacao]"


In [9]:
all_topics[gazeta][pg]['topics_infoMaritalk'][:5]

,Topic,Count,Name,Representation,Representative_Docs
0,-1,115,-1_coordenador_coordenacao_municipio_emergencia,"[coordenador, coordenacao, municipio, emergenc...","[coordenador, coordenador, coordenador]"
1,0,41,0_operacao_prefeito_viceprefeito_promocao,"[operacao, prefeito, viceprefeito, promocao, p...","[operacao, operacao, operacao]"
2,1,40,1_capitulo_privado_temporario_atribuicao,"[capitulo, privado, temporario, atribuicao, po...","[privado, capitulo, privado]"
3,2,36,2_acordo_entidade_decreto_situacao,"[acordo, entidade, decreto, situacao, modifica...","[entidade, acordo, entidade]"
4,3,29,3_autorizar_representante_qualquer_monitorar,"[autorizar, representante, qualquer, monitorar...","[autorizar, autorizar, autorizar]"


In [10]:
all_topics[gazeta][pg]['topics_infoZeroshot'][:5]

,Topic,Count,Name,Representation,Representative_Docs
0,0,37,Dispensas de Licitação,"[publico, informacao, resultado, modificacao, ...","[publico, publico, Publico]"
1,1,7,Contratações Emergenciais,"[emergencia, , , , , , , , , ]","[emergencia, emergencia, Emergencia]"
2,2,5,Aquisições Diretas,"[diretor, direto, direito, diretriz, , , , , , ]","[direto, diretor, diretor]"
3,3,71,-1_membro___,"[membro, , , , , , , , , ]","[membro, membro, membro]"
4,4,24,0_operacao_funcao_articulacao_legislacao,"[operacao, funcao, articulacao, legislacao, br...","[operacao, operacao, operacao]"


In [11]:
all_topics[gazeta][pg]['Ner']

[(periodo, 'PER'),
 (gestao convenio, 'LOC'),
 (modificacao, 'LOC'),
 (instituicao comite, 'ORG'),
 (Educacao, 'LOC'),
 (Camara Municipal, 'PER'),
 (Ministerio Publico, 'ORG'),
 (Brasil, 'LOC'),
 (Centro, 'LOC'),
 (Guarda, 'LOC'),
 (Secretaria Municipal, 'LOC'),
 (Secretaria Municipal, 'LOC'),
 (Centro, 'LOC'),
 (Centro, 'LOC')]

In [12]:
all_topics[gazeta][pg]['Ner_text']

[(Capítulo, 'MISC'),
 (Município de Salvador, 'LOC'),
 (Decreto, 'MISC'),
 (Lei, 'MISC'),
 (CAPÍTULO V DO COMITÊ DE INTEGRAÇÃO INSTITUCIONAL  Art. 15, 'MISC'),
 (Comitê de Integração Institucional, 'ORG'),
 (Município de Salvador, 'LOC'),
 (Covid-19, 'LOC'),
 (Comitê de Integração Institucional, 'ORG'),
 (I - Prefeito Municipal, 'MISC'),
 (II - Vice-Prefeito, 'MISC'),
 (Casa Civil, 'PER'),
 (IV - Chefe de Gabinete do Prefeito, 'MISC'),
 (V - Procurador-Geral do Município, 'MISC'),
 (Secretário Municipal da Saúde, 'PER'),
 (Secretário Municipal da Fazenda, 'LOC'),
 (Secretário Municipal de Promoção Social e Combate à Pobreza, 'PER'),
 (Secretário Municipal de Educação, 'PER'),
 (Secretário Municipal de Desenvolvimento e Urbanismo, 'LOC'),
 (XIV- Secretário Municipal de Ordem Pública, 'PER'),
 (Controlador-Geral do Município, 'LOC'),
 (Comitê de Integração Institucional, 'ORG'),
 (Presidente, 'MISC'),
 (Secretários Municipais, 'PER'),
 (Administração Indireta Municipal, 'ORG'),
 (Prefeit

In [15]:
print(all_topics[gazeta][pg]['Ner_mari_txt'])

  ```python
{
  "Person": ["Prefeito Municipal", "Vice-Prefeito", "Chefe da Casa Civil", "Chefe de Gabinete do Prefeito", "Procurador-Geral do Município", "Secretário Municipal da Saúde", "Secretário Municipal da Fazenda", "Secretário Municipal de Promoção Social e Combate à Pobreza", "Secretário Municipal de Educação", "Secretário Municipal de Infraestrutura", "Secretário Municipal de Gestão", "Secretário Municipal de Mobilidade", "Secretário Municipal de Desenvolvimento e Urbanismo", "Secretário Municipal de Ordem Pública", "Secretário Municipal de Comunicação", "Secretário-Geral de Articulação Comunitária e Prefeituras-Bairro", "Controlador-Geral do Município"],
  "Organization": ["Prefeitura Municipal de Salvador", "Centro de Coordenação de Operações"],
  "Location": ["Salvador"],
  "Date": ["19 de março de 2020", "perdurar durante o período da pandemia", "3 (três) meses"],
  "Time": ["durante o estado de emergência", "estado de calamidade pública decretados em função do Covid-19"]

In [17]:
print(all_topics[gazeta][pg]['Ner_mari'])

  ```python
{
    "Person": ["prever", "capitulo", "carater", "temporario", "excepcional", "perdurar", "durante", "periodo", "pandemia", "paragrafo", "autorizar", "prorrogacao", "oficio", "colaboracao", "fomento", "acordo", "cooperacao", "termos", "parceria", "contrato", "repasse", "contrato", "gestao", "convenio", "ajuste", "similar", "vencimento", "decretacao", "emergencia", "municipio", "salvador", "duracao", "Emergencia", "decretacao", "emergencia", "condicao", "avencadas", "observar", "aspecto", "procedimental", "legislacao", "pertinente", "ajuste", "prorrogar", "autorizar", "estabelecimento", "mediante", "decreto", "excepcional", "prestacao", "ajuste", "sofram", "modificacao", "prever", "justificar", "cenario", "emergencia", "acordo", "legislacao", "vigente", "capitulo", "comite", "integracao", "institucional", "reconhecido", "instituicao", "comite", "integracao", "institucional", "finalidade", "centralizar", "tomada", "decisao", "integrar", "alinhar", "governamental", "enfrentam